In [1]:
import cv2
import mss
import time
import pyautogui
import numpy as np
from pathlib import Path
from dataclasses import dataclass
from typing import Optional, Tuple

In [2]:
pyautogui.PAUSE = 0.1
pyautogui.FAILSAFE = True

In [3]:
@dataclass
class MatchResult:
    found: bool
    x: int = 0
    y: int = 0
    width: int = 0
    height: int = 0
    confidence: float = 0.0

    @property
    def center(self) -> Tuple[int, int]:
        return (self.x + self.width // 2, self.y + self.height // 2)


@dataclass
class ActionSequence:
    name: str
    templates: list[np.ndarray]
    template_names: list[str]

In [ ]:
class ScreenImageDetector:
    def __init__(self, confidence_threshold: float = 0.8):
        self.confidence_threshold = confidence_threshold
        self.sct = mss.mss()

    def capture_screen(self) -> np.ndarray:
        monitor_info = self.sct.monitors[0]
        screenshot = self.sct.grab(monitor_info)
        img = np.array(screenshot)
        return cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

    def load_template(self, template_path: str) -> np.ndarray:
        template = cv2.imread(template_path)
        if template is None:
            raise FileNotFoundError(f"Could not load template: {template_path}")
        return template

    def load_action_sequence(self, folder_path: str) -> ActionSequence:
        folder = Path(folder_path)
        if not folder.exists():
            raise FileNotFoundError(f"Folder not found: {folder_path}")
        
        png_files = sorted(folder.glob("*.png"))
        
        if not png_files:
            raise ValueError(f"No PNG files found in: {folder_path}")
        
        templates = []
        template_names = []
        
        for png_file in png_files:
            template = self.load_template(str(png_file))
            templates.append(template)
            template_names.append(png_file.stem)
        
        return ActionSequence(name=folder.name, templates=templates, template_names=template_names)

    def load_all_sequences(self, assets_folder: str = "assets") -> list[ActionSequence]:
        assets_path = Path(assets_folder)
        if not assets_path.exists():
            raise FileNotFoundError(f"Assets folder not found: {assets_folder}")
        
        sequences = []
        for subfolder in sorted(assets_path.iterdir()):
            if subfolder.is_dir():
                try:
                    sequence = self.load_action_sequence(str(subfolder))
                    sequences.append(sequence)
                    print(f"Loaded sequence '{sequence.name}' with {len(sequence.templates)} actions")
                except ValueError as e:
                    print(f"Skipping {subfolder.name}: {e}")
        
        return sequences

    def find_image(self, template: np.ndarray, screenshot: Optional[np.ndarray] = None, use_grayscale: bool = True) -> MatchResult:
        if screenshot is None:
            screenshot = self.capture_screen()

        if use_grayscale:
            screenshot_gray = cv2.cvtColor(screenshot, cv2.COLOR_BGR2GRAY)
            template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
            result = cv2.matchTemplate(screenshot_gray, template_gray, cv2.TM_CCOEFF_NORMED)
        else:
            result = cv2.matchTemplate(screenshot, template, cv2.TM_CCOEFF_NORMED)

        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        h, w = template.shape[:2]

        if max_val >= self.confidence_threshold:
            return MatchResult(found=True, x=max_loc[0], y=max_loc[1], width=w, height=h, confidence=max_val)
        
        return MatchResult(found=False, confidence=max_val)

    def click_at(self, x: int, y: int, clicks: int = 1, button: str = "left"):
        mon = self.sct.monitors[0]
        abs_x = x + mon["left"]
        abs_y = y + mon["top"]
        pyautogui.click(abs_x, abs_y, clicks=clicks, button=button)

    def find_and_click(self, template: np.ndarray, clicks: int = 1, button: str = "left", offset: Tuple[int, int] = (0, 0)) -> MatchResult:
        match = self.find_image(template)

        if match.found:
            center_x, center_y = match.center
            click_x = center_x + offset[0]
            click_y = center_y + offset[1]
            self.click_at(click_x, click_y, clicks=clicks, button=button)

        return match

    def execute_sequence(self, sequence: ActionSequence, step_delay: float = 0.5, timeout_per_step: float = 10.0, check_interval: float = 0.3) -> bool:
        print(f"  Executing sequence: {sequence.name}")
        
        for i, (template, name) in enumerate(zip(sequence.templates, sequence.template_names)):
            start_time = time.time()
            found = False
            
            while time.time() - start_time < timeout_per_step:
                match = self.find_and_click(template)
                if match.found:
                    print(f"    [{i+1}/{len(sequence.templates)}] Clicked '{name}' at {match.center}")
                    found = True
                    time.sleep(step_delay)
                    break
                time.sleep(check_interval)
            
            if not found:
                print(f"    [{i+1}/{len(sequence.templates)}] Timeout waiting for '{name}'")
                return False
        
        return True

    def find_first_sequence(self, sequences: list[ActionSequence], screenshot: Optional[np.ndarray] = None) -> Optional[ActionSequence]:
        if screenshot is None:
            screenshot = self.capture_screen()
        
        for sequence in sequences:
            if sequence.templates:
                match = self.find_image(sequence.templates[0], screenshot)
                if match.found:
                    return sequence
        
        return None

In [5]:
def monitor_and_execute(detector: ScreenImageDetector, sequences: list[ActionSequence], duration: float = 300.0, check_interval: float = 1.0, cooldown: float = 2.0, step_delay: float = 0.5):
    start_time = time.time()
    execution_count = 0
    
    print(f"Monitoring for {duration}s...")
    print(f"Loaded {len(sequences)} sequences:")
    for seq in sequences:
        print(f"  - {seq.name}: {seq.template_names}")
    print("---")
    
    try:
        while time.time() - start_time < duration:
            screenshot = detector.capture_screen()
            sequence = detector.find_first_sequence(sequences, screenshot)
            
            if sequence:
                execution_count += 1
                print(f"[{time.strftime('%H:%M:%S')}] Found sequence '{sequence.name}' (#{execution_count})")
                
                success = detector.execute_sequence(sequence, step_delay=step_delay)
                
                if success:
                    print(f"  Completed successfully!")
                else:
                    print(f"  Sequence incomplete")
                
                time.sleep(cooldown)
            else:
                time.sleep(check_interval)
                
    except KeyboardInterrupt:
        print("\nStopped by user")
    
    print(f"---\nFinished. Total executions: {execution_count}")

In [ ]:
detector = ScreenImageDetector(confidence_threshold=0.8)
sequences = detector.load_all_sequences("../assets")

Loaded sequence 'auto-challenge' with 3 actions


In [ ]:
monitor_and_execute(detector=detector, sequences=sequences, duration=300.0, check_interval=1.0, cooldown=5.0, step_delay=4.0)

Monitoring for 300.0s...
Loaded 1 sequences:
  - auto-challenge: ['action-1', 'action-2', 'action-3']
---
[22:57:43] Found sequence 'auto-challenge' (#1)
  Executing sequence: auto-challenge
    [1/3] Clicked 'action-1' at (2531, 1750)
    [2/3] Clicked 'action-2' at (2428, 1750)
    [3/3] Clicked 'action-3' at (2643, 1693)
  Completed successfully!
[22:58:08] Found sequence 'auto-challenge' (#2)
  Executing sequence: auto-challenge
    [1/3] Clicked 'action-1' at (2531, 1750)
    [2/3] Clicked 'action-2' at (2428, 1750)
    [3/3] Clicked 'action-3' at (2643, 1693)
  Completed successfully!
[22:59:03] Found sequence 'auto-challenge' (#3)
  Executing sequence: auto-challenge
    [1/3] Clicked 'action-1' at (2531, 1750)
    [2/3] Clicked 'action-2' at (2428, 1750)
    [3/3] Clicked 'action-3' at (2643, 1823)
  Completed successfully!
[22:59:51] Found sequence 'auto-challenge' (#4)
  Executing sequence: auto-challenge
    [1/3] Clicked 'action-1' at (2214, 1391)
    [2/3] Timeout waiting 